In [1]:
import gradio as gr
import whisper
from gtts import gTTS
import requests
import tempfile
import base64
import os
import re

In [2]:
#Load environment variables from .env
from dotenv import load_dotenv
load_dotenv()
API_KEY = os.getenv("OPENROUTER_API_KEY")

In [3]:
# Load Whisper model
whisper_model = whisper.load_model("base")

In [4]:
def initialize_messages():
    return [{
        "role": "system",
        "content": (
            "You are an AI teacher built for classrooms. Answer with examples, analogies, and help students learn. "
            "Also, suggest relevant images and YouTube video links where appropriate."
            "Respond without using symbols like @, #, %, $, &, *, etc."

        )
    }]

messages_prmt = initialize_messages()

In [5]:
# Clean unwanted symbols from text
def clean_text(text):
    # Remove symbols like @, #, $, %, ^, &, *, etc.
    return re.sub(r'[^\w\s.,!?]', '', text)

In [6]:
# Convert text to base64 audio
def text_to_base64_audio(text):
    cleaned_text = clean_text(text)
    tts = gTTS(cleaned_text)
    with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3") as temp_audio:
        tts.save(temp_audio.name)
        audio_data = open(temp_audio.name, "rb").read()
    os.remove(temp_audio.name)
    return base64.b64encode(audio_data).decode("utf-8")

In [7]:
# Get YouTube video search link
def get_youtube_search_link(query):
    return f"https://www.youtube.com/results?search_query={query.replace(' ', '+')}"

In [8]:
# Get Unsplash or DuckDuckGo image search link
def get_image_search_link(query):
    return f"https://duckduckgo.com/?q={query.replace(' ', '+')}&iax=images&ia=images"

In [9]:
# Main logic
def ai_teacher(audio_input, text_input):
    global messages_prmt

    user_query = ""

    # Handle voice
    if audio_input:
        transcription = whisper_model.transcribe(audio_input)["text"]
        user_query = transcription.strip()
    elif text_input:
        user_query = text_input.strip()
    else:
        return "<p>Please provide either voice or text input.</p>"

    messages_prmt.append({"role": "user", "content": user_query})

    # Call OpenRouter API
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }
    data = {
        "model": "deepseek/deepseek-r1-0528:free",
        "messages": messages_prmt
    }
    response = requests.post("https://openrouter.ai/api/v1/chat/completions", headers=headers, json=data)
    result = response.json()

    if "choices" not in result:
        return "<p>Error: OpenRouter did not return a response.</p>"

    bot_reply = result["choices"][0]["message"]["content"]
    messages_prmt.append({"role": "assistant", "content": bot_reply})

    # Audio conversion
    b64_audio = text_to_base64_audio(bot_reply)

    # Search links
    youtube_link = get_youtube_search_link(user_query)
    image_link = get_image_search_link(user_query)

    # Format readable HTML output
    html = f"""
    <div style="font-family: Arial; line-height: 1.6;">
        <p><strong>🧑‍🎓 You asked:</strong> {user_query}</p>
        <p><strong>🧠 AI explains:</strong></p>
        <div style="background-color:#000000; padding: 12px; border-left: 4px solid #4CAF50; white-space: pre-wrap;">
            {bot_reply}
        </div>
        <p><strong>🔉 Listen to the answer:</strong></p>
        <audio autoplay controls style="width: 100%;">
            <source src="data:audio/mp3;base64,{b64_audio}" type="audio/mp3">
            Your browser does not support the audio element.
        </audio>
        <p><strong>🖼 Related Images:</strong> <a href="{image_link}" target="_blank">View Image Search</a></p>
        <p><strong>🎥 Related Videos:</strong> <a href="{youtube_link}" target="_blank">Watch on YouTube</a></p>
    </div>
    """

    return html

In [10]:
# Gradio Interface
iface = gr.Interface(
    fn=ai_teacher,
    inputs=[
    gr.Audio(type="filepath", label="Speak your question (optional)"),
    gr.Textbox(placeholder="Or type your question here...", label="Text input (optional)")
    ],
    outputs=gr.HTML(),
    title="AI Classroom Voice & Text Teacher",
    description="Ask a question by voice or text. Get a spoken answer, readable text, and links to visuals.",
)

iface.launch(share=True)

* Running on local URL:  http://127.0.0.1:7861

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.
